In [1]:
from build_dataset import MRILesionDatasetBuilder

# Build the dataset for the lesion inpaiting model
- Train / test split
- Filter out slices with no lesions
- Include SHIFTS and WMH2017 datasets

### VH

In [ ]:
data_folder="/home/benet/data"
input_folder="VH"
output_folder="lesion2D_VH-SHIFTS-WMH2017"
folders=["train", "test"]
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
train_split=0.7
seed = 17844

dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed)
dataset_builder.build_dataset()

Skipping empty mask for train 640 at slice 95
Skipping empty mask for train 640 at slice 96
Skipping empty mask for train 640 at slice 97
Skipping empty mask for train 644 at slice 86
Skipping empty mask for train 644 at slice 87
Skipping empty mask for train 644 at slice 88
Skipping empty mask for train 644 at slice 89
Skipping empty mask for train 644 at slice 90
Skipping empty mask for train 644 at slice 91
Skipping empty mask for train 644 at slice 92
Skipping empty mask for train 644 at slice 93
Skipping empty mask for train 644 at slice 94
Skipping empty mask for train 644 at slice 95
Skipping empty mask for train 644 at slice 96
Skipping empty mask for train 644 at slice 97
Skipping empty mask for train 658 at slice 85
Skipping empty mask for train 658 at slice 86
Skipping empty mask for train 658 at slice 87
Skipping empty mask for train 658 at slice 88
Skipping empty mask for train 658 at slice 89
Skipping empty mask for train 658 at slice 90
Skipping empty mask for train 658 

### SHIFTS

In [3]:
data_folder="/home/benet/data"
input_folder="SHIFTS_preprocessedMNI" ###
output_folder="lesion2D_VH-SHIFTS-WMH2017"
folders=["dev_in", "dev_out", "eval_in", "train"] ###
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
seed = 17844

dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed)
dataset_builder.build_dataset()

Skipping empty mask for dev_in 3 at slice 85
Skipping empty mask for dev_in 3 at slice 86
Skipping empty mask for dev_in 3 at slice 87
Skipping empty mask for dev_in 3 at slice 88
Skipping empty mask for dev_in 3 at slice 89
Skipping empty mask for dev_in 3 at slice 90
Skipping empty mask for dev_in 3 at slice 91
Skipping empty mask for dev_in 3 at slice 92
Skipping empty mask for dev_in 3 at slice 93
Skipping empty mask for dev_in 3 at slice 94
Skipping empty mask for dev_in 3 at slice 95
Skipping empty mask for dev_out 13 at slice 92
Skipping empty mask for dev_out 13 at slice 93
Skipping empty mask for dev_out 13 at slice 94
Skipping empty mask for dev_out 13 at slice 95
Skipping empty mask for dev_out 13 at slice 96
Skipping empty mask for dev_out 13 at slice 97
Skipping empty mask for dev_out 24 at slice 87
Skipping empty mask for dev_out 24 at slice 88
Skipping empty mask for dev_out 24 at slice 89
Skipping empty mask for dev_out 24 at slice 90
Skipping empty mask for eval_in 20 

### WMH2017

In [4]:
data_folder="/home/benet/data"
input_folder="WMH2017_preprocessedMNI" ###
output_folder="lesion2D_VH-SHIFTS-WMH2017"
folders=None ###
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
seed = 17844

dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed)
dataset_builder.build_dataset()

Skipping empty mask for None 54 at slice 91
Total empty masks skipped: 1
In the preprocessed folder: Total examples: 779, train examples: 545 (69.96%), test examples: 234 (30.04%)
In the hole dataset: Total examples: 2620, train examples: 1801 (68.74%), test examples: 819 (31.26%)
